In [12]:
(def lis [1 2 3 4 5 6])

#'user/lis

In [13]:
;; reduce <function> <inital-state> <iterable>

(reduce + 0 lis)

21

In [14]:
;; reduce <function> <iterable>
;; assumes the inital state is 0
(reduce + lis)

21

In [15]:
;; implementing map using reduce

(defn my-map [f xs]
    (reduce
     (fn [s x] (conj s (f x)))
     []
     xs))

#'user/my-map

In [19]:
(my-map (partial * 2) lis)

[2 4 6 8 10 12]

In [36]:
(my-map (fn [x] (if (odd? x) 1 0)) lis)

[1 0 1 0 1 0]

In [70]:
(def B 1e9)

(def social-network
    {
     :members [ 
               {
                 :id 1000
                 :name "Jack"
                 :age 18
                 :hobbies ["sport" "reading"]
                 }
                {
                 :id 1001
                 :name "Joe"
                 :age 22
                 :occupation "electrition"
                 }
                {
                 :id 1002
                 :name "Elon"
                 :occupation "ceo"
                 :hobbies ["space" "coding"]
                 }
               ]
     :companies [
                 {
                  :name "GM"
                  :size 157000
                  :market-cap (* 55 B)
                  }
                 {
                  :name "Twitter"
                  :size 7000
                  :market-cap (* 41 B)
                  }
                 {
                  :name "Tesla"
                  :size 2500
                  :market-cap (* 30 B)
                  }
                 {
                  :name "Spotify"
                  :size 6000
                  :market-cap (* 15 B)
                  }
                 ]
      :connections [
                    {
                     :person-id 1002
                     :company "Tesla"
                     :type :owns
                     }
                    {
                     :person-id 1000
                     :company "Twitter"
                     :type :use
                     }
                    {
                     :person-id 1002
                     :company "Twitter"
                     :type :owns
                     }
                    {
                     :person-id 1001
                     :company "GM"
                     :type :works
                     }
                    ]
    }
)

#'user/social-network

In [71]:
;; compute the average value per employee
;; using for-loop
(let [companies (social-network :companies)
      total-size (apply + (for [company companies] (:size company)))
      total-market-cap (apply + (for [company companies] (:market-cap company)))]
    (/ total-market-cap total-size))

817391.304347826

In [72]:
;; using map reduce

(/ (reduce + (map :market-cap (:companies social-network)))
   (reduce + (map :size (:companies social-network)))
)

817391.304347826

In [73]:
;; using reduce 
(as-> (reduce (fn [state company] {:total-market-cap (+ (:total-market-cap state) (:market-cap company))
                                   :total-size (+ (:total-size state) (:size company))})
        {:total-market-cap 0 :total-size 0}
        (:companies social-network))
    $
    (/ (:total-market-cap $)
       (:total-size $))
    )

817391.304347826

In [74]:
(defn add-by-attr [state attr company]
    (update state attr (fn [x] (+ x (get company attr)))))

#'user/add-by-attr

In [75]:
(add-by-attr {:market-cap 100 :size 100} :market-cap {:market-cap 3 :size 5})

{:market-cap 103, :size 100}

In [76]:
;; using reduce 
(as-> (reduce (fn [state company] (-> state (add-by-attr :market-cap company) (add-by-attr :size company)))
        {:market-cap 0 :size 0}
        (:companies social-network))
    $
    (/ (:market-cap $)
       (:size $))
    )
    

817391.304347826

In [96]:
(defn id-of [name]
    (->> social-network
        (:members)
        (filter (fn [member] (= (:name member) name)))
        (first)
        (:id)))

(defn connections-of [id]
    (->> social-network
        (:connections)
        (filter (fn [conn] (= (:person-id conn) id)))
        ))

(defn info-of [company-name]
    (->> social-network
        (:companies)
        (filter (fn [company] (= (:name company) company-name)))
        (first)))

#'user/info-of

In [97]:
(id-of "Elon")

1002

In [98]:
(info-of "Twitter")

{:name "Twitter", :size 7000, :market-cap 4.1E10}

In [99]:
(->> "Elon" (id-of) (connections-of) (map :company) (map info-of))

({:name "Tesla", :size 2500, :market-cap 3.0E10} {:name "Twitter", :size 7000, :market-cap 4.1E10})